In [1]:
#Import

#pandas
import pandas as pd
from pandas import Series,DataFrame

#numpy
import numpy as np

In [2]:
#导入数据

card_train_feature=pd.read_csv('input/card_train.csv')
card_train_feature.head()

,id,amount,remainder,date,hour_00_01,hour_06_07,hour_08_09,hour_11_13,hour_17_21,hour_22_23,...,shower,market,canteen,f_ten_canteen,other_canteen,f_ten_place,other_place,winter_vacation,summer_vacation,on_school
0,1006,0.50,124.90,09-01-2013,1,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,1
1,1968,0.10,200.14,09-01-2013,1,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,1
2,1406,0.01,374.42,09-01-2013,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
3,1406,0.60,373.82,09-01-2013,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
4,13554,0.50,522.37,09-01-2013,1,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,1


In [3]:
#card_train_feature.info()

In [3]:
#根据id聚合求解各个特征的产生的总数

count_columns=list(card_train_feature.columns)[4:]

#count_columns
count_pivot=card_train_feature.pivot_table(index='id',values=count_columns,aggfunc=np.count_nonzero)

In [4]:
#根据id聚合求解各个特征均值(各特征消费次数占比)

a=['id','date']
mean_columns=[x for x in list(card_train_feature.columns) if x not in a]
mean_pivot=card_train_feature.pivot_table(index='id',values=mean_columns,aggfunc=np.mean)

#对列名重命名
mean_pivot.columns=[x+'_mean' for x in list(mean_pivot.columns)]

#连接两表
card_train_feature_f=mean_pivot.join(count_pivot)

#计算消费总额与余额总额
card_train_feature_s=card_train_feature.groupby('id')['amount','remainder'].sum()
card_train_feature_f=card_train_feature_f.join(card_train_feature_s)

In [5]:
#获得id关于每一列的消费总额

def get_colunms_sum(df):
    a=list(df.columns)[4:]
    columns_sum=DataFrame(df.groupby('id')['id'].count())#初始化表格大小
    columns_sum.columns=['id_count']
    for x in range(len(a)):
            columns_sum_x=DataFrame(df.loc[df[a[x]]==1].groupby('id')['amount'].sum())
            columns_sum_x.columns=[a[x]+'_amount_sum']
            columns_sum=columns_sum.join(columns_sum_x)
            x+=1
    columns_sum.drop('id_count',axis=1,inplace=True)
    return columns_sum

#运用定义的函数
card_columns_sum=get_colunms_sum(card_train_feature)

#连接总表
card_train_feature_f=card_train_feature_f.join(card_columns_sum)

In [6]:
#定义id关于每一列的消费均值(每次)

def get_colunms_mean(df_sum,df_count):
    a=list(df_count.columns)
    columns_mean=DataFrame(df_count[a[0]])#初始化表格大小
    columns_mean.columns=['id_count']
    for x in range(len(a)):
        columns_count_x=df_count[a[x]].values
        columns_sum_x=df_sum[a[x]+'_amount_sum'].values
        columns_mean[a[x]+'_amount_mean']=np.divide.reduce([columns_sum_x,columns_count_x],axis=0)
        x+=1
    columns_mean.drop('id_count',axis=1,inplace=True)
    return columns_mean

#运用定义的函数
card_columns_mean=get_colunms_mean(card_columns_sum,count_pivot)

#连接总表
card_train_feature_f=card_train_feature_f.join(card_columns_mean)

In [7]:
#定义id关于每一列的消费额占总额的比例

def get_colunms_amount_rate(df_sum,df):
    a=list(df_sum.columns)
    columns_amount_rate=DataFrame(df_sum[a[0]])#初始化表格大小
    columns_amount_rate.columns=['id_count']
    amount_sum=df['amount'].values
    for x in range(len(a)):
        columns_amountsum_x=df_sum[a[x]].values
        columns_amount_rate[a[x]+'_rate']=np.divide.reduce([columns_amountsum_x,amount_sum],axis=0)
        x+=1
    columns_amount_rate.drop('id_count',axis=1,inplace=True)
    return columns_amount_rate

#运用定义的函数
card_columns_amount_rate=get_colunms_amount_rate(card_columns_sum,card_train_feature_f)

#连接总表
card_train_feature_f=card_train_feature_f.join(card_columns_amount_rate)

In [8]:
#获得id关于每一列的消费的最大值

def get_colunms_amount_max(df):
    a=list(df.columns)[4:]
    columns_amount_max=DataFrame(df.groupby('id')['id'].count())#初始化表格大小
    columns_amount_max.columns=['id_count']
    for x in range(len(a)):
            columns_max_x=DataFrame(df.loc[df[a[x]]==1].groupby('id')['amount'].max())
            columns_max_x.columns=[a[x]+'_amount_max']
            columns_amount_max=columns_amount_max.join(columns_max_x)
            x+=1
    columns_amount_max.drop('id_count',axis=1,inplace=True)
    return columns_amount_max

#运用定义的函数
columns_amount_max=get_colunms_amount_max(card_train_feature)

#连接总表
card_train_feature_f=card_train_feature_f.join(columns_amount_max)

In [9]:
#获得id关于每一列的消费的最小值

def get_colunms_amount_min(df):
    a=list(df.columns)[4:]
    columns_amount_min=DataFrame(df.groupby('id')['id'].count())#初始化表格大小
    columns_amount_min.columns=['id_count']
    for x in range(len(a)):
            columns_min_x=DataFrame(df.loc[df[a[x]]==1].groupby('id')['amount'].min())
            columns_min_x.columns=[a[x]+'_amount_min']
            columns_amount_min=columns_amount_min.join(columns_min_x)
            x+=1
    columns_amount_min.drop('id_count',axis=1,inplace=True)
    return columns_amount_min

#运用定义的函数
columns_amount_min=get_colunms_amount_min(card_train_feature)

#连接总表
card_train_feature_f=card_train_feature_f.join(columns_amount_min)

In [10]:
#获得id关于每一列的消费的标准方差

def get_colunms_amount_std(df):
    a=list(df.columns)[4:]
    columns_amount_std=DataFrame(df.groupby('id')['id'].count())#初始化表格大小
    columns_amount_std.columns=['id_count']
    for x in range(len(a)):
            columns_std_x=DataFrame(df.loc[df[a[x]]==1].groupby('id')['amount'].std())
            columns_std_x.columns=[a[x]+'_amount_std']
            columns_amount_std=columns_amount_std.join(columns_std_x)
            x+=1
    columns_amount_std.drop('id_count',axis=1,inplace=True)
    return columns_amount_std

#运用定义的函数
columns_amount_std=get_colunms_amount_std(card_train_feature)

#连接总表
card_train_feature_f=card_train_feature_f.join(columns_amount_std)

In [11]:
#获得id关于每一列的消费的中位数

def get_colunms_amount_median(df):
    a=list(df.columns)[4:]
    columns_amount_median=DataFrame(df.groupby('id')['id'].count())#初始化表格大小
    columns_amount_median.columns=['id_count']
    for x in range(len(a)):
            columns_median_x=DataFrame(df.loc[df[a[x]]==1].groupby('id')['amount'].median())
            columns_median_x.columns=[a[x]+'_amount_median']
            columns_amount_median=columns_amount_median.join(columns_median_x)
            x+=1
    columns_amount_median.drop('id_count',axis=1,inplace=True)
    return columns_amount_median

#运用定义的函数
columns_amount_median=get_colunms_amount_median(card_train_feature)

#连接总表
card_train_feature_f=card_train_feature_f.join(columns_amount_median)

In [12]:
#计算单笔消费额的最大、最小值、中位数、方差
a='amount'
card_train_feature_f[a+'_max']=card_train_feature.groupby('id')[a].max()
card_train_feature_f[a+'_min']=card_train_feature.groupby('id')[a].min()
card_train_feature_f[a+'_median']=card_train_feature.groupby('id')[a].median()
card_train_feature_f[a+'_std']=card_train_feature.groupby('id')[a].std()

#计算消费余额的最大、最小值、中位数、方差

b='remainder'
card_train_feature_f[b+'_max']=card_train_feature.groupby('id')[b].max()
card_train_feature_f[b+'_min']=card_train_feature.groupby('id')[b].min()
card_train_feature_f[b+'_median']=card_train_feature.groupby('id')[b].median()
card_train_feature_f[b+'_std']=card_train_feature.groupby('id')[b].std()

#计算消费总笔数

card_train_feature_f['amount_count']=card_train_feature.groupby('id')['amount'].count()

In [13]:
#每个id的活跃天数

card_train_date=DataFrame(card_train_feature.groupby(['id','date'])['id'].count())
card_train_date.to_csv('input/card_train_day.csv',index=True)
card_train_day=pd.read_csv('input/card_train_day.csv')
card_train_day_sum=DataFrame(card_train_day.groupby('id')['id.1'].count())
card_train_day_sum.columns=['card_day_sum']

#平均每天消费总额

card_train_feature_f['card_day_sum']=card_train_day_sum['card_day_sum']
card_train_feature_f['amount_mean_day']=card_train_feature_f['amount']/card_train_feature_f['card_day_sum']

In [14]:
#周末消费均值减去非周末消费均值(每次)

card_train_feature_f['rd_wd_difference']=card_train_feature_f['resting_day_amount_mean']-card_train_feature_f['working_day_amount_mean']


In [15]:
#“圈存转账”“卡充值”“支付领取”三种充值类特征相加

a=card_train_feature_f['card_recharge_amount_sum'].add(card_train_feature_f['load_tansfer_amount_sum'],fill_value=0)
card_train_feature_f['three_recharge_amount_sum']=a.add(card_train_feature_f['payment_collection_amount_sum'],fill_value=0)

#卡充值总额/活跃天数

card_train_feature_f['recharge_day_mean']=card_train_feature_f['three_recharge_amount_sum'].divide(card_train_feature_f['card_day_sum'],fill_value=0)


In [16]:
#单笔消费额在0-2.5，2.5-5，5-10，10-20的总笔数/活跃天数，消费笔数，活跃天数

def get_amount_count(df1,df2):
    a=[2.5,5,10,20]
    for x in range(len(a)):
        if x==0:
            #消费总笔数
            amount_count_x=DataFrame(df1.loc[df1['amount']<=a[x]].groupby('id')['amount'].count())
            amount_count_x.columns=['amount_slice_0_'+str(a[x])+'_count']
            #活跃天数
            amount_day=DataFrame(df1.loc[df1['amount']<=a[x]].groupby(['id','date'])['id'].count())                      
            amount_day.to_csv('input/card_train_amount_day.csv',index=True)
            amount_day=pd.read_csv('input/card_train_amount_day.csv')
            slice_day_sum_x=DataFrame(amount_day.groupby('id')['id.1'].count())
            slice_day_sum_x.columns=['day_sum']
            #df2['amount_slice_0_'+str(a[x])]=amount_count_x['amount'].divide(slice_day_sum_x['day_sum'],fill_value=0)
            #活跃笔数/活跃天数
            amount_count_x['amount_slice_0_'+str(a[x])+'_rate_day']=amount_count_x['amount_slice_0_'+str(a[x])+'_count'].divide(slice_day_sum_x['day_sum'],fill_value=0)
            amount_count_x['amount_slice_0_'+str(a[x])+'_day_sum']=slice_day_sum_x['day_sum']
            df2=df2.join(amount_count_x)
        else:
            #消费总笔数
            b=df1.loc[df1['amount']>a[x-1]]
            amount_count_x=DataFrame(b.loc[b['amount']<=a[x]].groupby('id')['amount'].count())
            amount_count_x.columns=['amount_slice_'+str(a[x-1])+'_'+str(a[x])+'_count']               
            #活跃天数
            amount_day=DataFrame(b.loc[b['amount']<=a[x]].groupby(['id','date'])['id'].count()) 
            amount_day.to_csv('input/card_train_amount_day.csv',index=True)
            amount_day=pd.read_csv('input/card_train_amount_day.csv')
            slice_day_sum_x=DataFrame(amount_day.groupby('id')['id.1'].count())
            slice_day_sum_x.columns=['day_sum']
            #活跃天数
            #df2['amount_slice_'+str(a[x-1])+'_'+str(a[x])]=amount_count_x['amount'].divide(slice_day_sum_x['day_sum'],fill_value=0)
            amount_count_x['amount_slice_'+str(a[x-1])+'_'+str(a[x])+'_rate_day']=amount_count_x['amount_slice_'+str(a[x-1])+'_'+str(a[x])+'_count'].divide(slice_day_sum_x['day_sum'],fill_value=0)
            amount_count_x['amount_slice_'+str(a[x-1])+'_'+str(a[x])+'_day_sum']=slice_day_sum_x['day_sum']
            df2=df2.join(amount_count_x)
            x+=1
    return df2

In [17]:
#运用函数

card_train_feature_f=get_amount_count(card_train_feature,card_train_feature_f)

In [20]:
#单笔消费额在0-2.5，2.5-5，5-10，10-20的活跃笔数占比，活跃天数占比

def get_amount_count1(df):
    a=[0,2.5,5,10,20]
    for x in range(len(a)-1):
        df['amount_slice_'+str(a[x])+'_'+str(a[x+1])+'_count_rate']=df['amount_slice_'+str(a[x])+'_'+str(a[x+1])+'_count'].divide(df['amount_count'],fill_value=0)
        df['amount_slice_'+str(a[x])+'_'+str(a[x+1])+'_day_rate']=df['amount_slice_'+str(a[x])+'_'+str(a[x+1])+'_day_sum'].divide(df['card_day_sum'],fill_value=0)       
    return df 



In [22]:
#运用函数

card_train_feature_f=get_amount_count1(card_train_feature_f)

In [25]:
#每天消费总额在0-10，10-20，20-30，30-40,40-50活跃天数

def get_amount_count2(df1,df2):
    a=[10,20,30,40,50]
    for x in range(len(a)):
        if x==0:
            #消费笔数
            amount_sum=DataFrame(df1.groupby(['id','date'])['amount'].sum())
            amount_sum.to_csv('input/card_train_amount_sum.csv',index=True)
            amount_sum_x=pd.read_csv('input/card_train_amount_sum.csv')
            slice_sum_x=DataFrame(amount_sum_x.loc[amount_sum_x['amount']<=a[x]].groupby('id')['amount'].count())
            slice_sum_x.columns=['amount_slice_day_0_'+str(a[x])+'_count']
            #df2['amount_slice_0_'+str(a[x])+'_all']=slice_sum_x['amount'].divide(df2['card_day_sum'],fill_value=0)
            df2=df2.join(slice_sum_x)
        else:
            #消费笔数
            amount_sum=DataFrame(df1.groupby(['id','date'])['amount'].sum())
            amount_sum.to_csv('input/card_train_amount_sum.csv',index=True)
            amount_sum_x=pd.read_csv('input/card_train_amount_sum.csv')
            b=amount_sum_x.loc[amount_sum_x['amount']>a[x-1]]
            slice_sum_x=DataFrame(b.loc[b['amount']<=a[x]].groupby('id')['amount'].count())
            slice_sum_x.columns=['amount_slice_day_'+str(a[x-1])+'_'+str(a[x])+'_count']
            #df2['amount_slice_'+str(a[x-1])+'_'+str(a[x])+'_all']=slice_sum_x['amount'].divide(df2['card_day_sum'],fill_value=0)
            df2=df2.join(slice_sum_x)
            x+=1
    return df2
            


In [26]:
#运用函数

card_train_feature_f=get_amount_count2(card_train_feature,card_train_feature_f)

In [28]:
#每天消费总额在0-10，10-20，20-30，30-40,40-50活跃天数/活跃天数
def get_amount_count3(df):
    a=[0,10,20,30,40,50]
    for x in range(len(a)-1):
        df['amount_slice_day_'+str(a[x])+'_'+str(a[x+1])+'day_rate']=df['amount_slice_day_'+str(a[x])+'_'+str(a[x+1])+'_count'].divide(df['card_day_sum'],fill_value=0)
    return df

In [29]:
#运用函数

card_train_feature_f=get_amount_count3(card_train_feature_f)

In [82]:
#card_train_feature_f.drop('amount_slice_10_20_sum',axis=1,inplace=True)

In [30]:
card_train_feature_f.head()

,POS_recharge_mean,amount_mean,bus_mean,canteen_mean,card_recharge_mean,f_ten_canteen_mean,f_ten_place_mean,hospital_mean,hour_00_01_mean,hour_02_05_mean,...,amount_slice_day_0_10_count,amount_slice_day_10_20_count,amount_slice_day_20_30_count,amount_slice_day_30_40_count,amount_slice_day_40_50_count,amount_slice_day_0_10day_rate,amount_slice_day_10_20day_rate,amount_slice_day_20_30day_rate,amount_slice_day_30_40day_rate,amount_slice_day_40_50day_rate
id,,,,,,,,,,,,,,,,,,,,,
0,0.949264,7.830016,0.162029,0.206219,0.036007,0.062193,0.409165,0.0,0.040917,0.003273,...,82.0,52.0,20.0,8.0,2.0,0.440860,0.279570,0.107527,0.043011,0.010753
1,0.968610,8.784709,0.086697,0.387145,0.016442,0.046338,0.162930,0.0,0.019432,0.000000,...,29.0,38.0,32.0,19.0,2.0,0.207143,0.271429,0.228571,0.135714,0.014286
8,0.970428,8.388444,0.197665,0.048249,0.028016,0.048249,0.759533,0.0,0.009339,0.000000,...,15.0,130.0,84.0,14.0,2.0,0.053381,0.462633,0.298932,0.049822,0.007117
9,0.973231,9.033875,0.019758,0.140217,0.026769,0.140217,0.670491,0.0,0.005099,0.000000,...,16.0,67.0,83.0,26.0,24.0,0.059041,0.247232,0.306273,0.095941,0.088561
10,0.957882,7.191348,0.145608,0.239471,0.031288,0.095066,0.308063,0.0,0.001203,0.000000,...,91.0,56.0,24.0,4.0,4.0,0.435407,0.267943,0.114833,0.019139,0.019139


In [31]:
len(card_train_feature_f)

10860

In [34]:
card_train_feature_f.to_csv('input/card_train_feature_f.csv',index='id')

In [35]:
card_train_feature_f.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10860 entries, 0 to 32671
Columns: 336 entries, POS_recharge_mean to amount_slice_day_40_50day_rate
dtypes: float64(302), int64(34)
memory usage: 27.9 MB
